In [1]:
import numpy as np
import aocd

In [2]:
test_input = """###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############"""

In [3]:
def parse_input(data):
    return np.array([[x for x in line] for line in data.split("\n")])

In [4]:
dirs = {"u": [-1,0],
        "d": [1, 0],
        "r": [0, 1],
        "l": [0,-1]}

In [5]:
def turn_left(direction):
    # Define the 90-degree counterclockwise rotation matrix
    turn_left_matrix = np.array([[0, -1], [1, 0]])
    
    # Multiply the direction by the rotation matrix
    return tuple(np.dot(turn_left_matrix, direction))

def turn_right(direction):
    # Define the 90-degree clockwise rotation matrix
    turn_right_matrix = np.array([[0, 1], [-1, 0]])
    
    # Multiply the direction by the rotation matrix
    return tuple(np.dot(turn_right_matrix, direction))

In [6]:
d = dirs["r"]

In [7]:
turn_left(d)

(np.int64(-1), np.int64(0))

In [8]:
turn_right(d) == dirs["d"]

False

In [9]:
def make_stringmap(arr):
    stringmap = []
    for line in arr:
        stringmap.append("".join(line))
    stringmap = "\n".join(stringmap)
    return stringmap

Had lots of issues and removed a ton of mess. BFS as with the hiking trails does not work. Recursion does not work. I have to somehow implement Dijkstra or A*. Did a ton of reading and sketching... Got up with the following: 

heapq library makes a list into a sort of priority queue, where heappop pops the tuple with lowest first value, meaning we can at any given point inspect the next step with the lowest cost. 


In [10]:
import heapq

In [11]:
#HEAPQ IS SUPER USEFUL HERE

In [20]:
def dijkstra(arr):
    start = np.argwhere(arr == "S")[0]
    visited = set()
    q = [(0, start, [0,1])] #cost, pos, direction
    while q:
        cost, loc, direction = heapq.heappop(q) #retrieve the lowest cost next step from the queue for checking
        if (tuple(loc), tuple(direction)) in visited: #if we have already been here, we are on the way back from a bad route and need to ignore the loc
            continue #we have done this
        if arr[loc[0], loc[1]] == "E": #if we stumble upon the target tile, we have found the (or at least one) lowest cost route.
            #arrived, we return cost
            return cost, visited
        visited.add((tuple(loc), tuple(direction))) #we record our current position and direction
        one_ahead = [loc[0] + direction[0], loc[1]+direction[1]] #look at the space one ahead of us:
        if arr[one_ahead[0], one_ahead[1]] != "#":
            heapq.heappush(q, (cost+1, [one_ahead[0], one_ahead[1]], direction)) #if the space one ahead of us is not a wall, it is an empty space or the end, and we need to explore it for a cost of 1
        #we also need to explore what happens if we turn left and right, for a cost of 1000 each.
        heapq.heappush(q, (cost+1000, loc, turn_left(direction)))
        heapq.heappush(q, (cost+1000, loc, turn_right(direction)))
        
        

In [24]:
cost, locs = dijkstra(parse_input(puzzle_input))

In [25]:
cost

82460

In [ ]:
# aocd.submit(cost)

In [26]:
max_cost = cost #this can be the exit criterium for recursion

In [34]:
from collections import defaultdict

In [49]:
def dijkstra_max_cost(arr, max_cost):
    start = np.argwhere(arr == "S")[0]
    all_paths = []
    visited = {}
    nodes = defaultdict(set) #collect all tiles with the connected tiles on correct paths
    q = [(0, [[start[0],start[1]]], [0,1], None)] #cost, pos, direction
    while q:
        cost, path, direction, prev = heapq.heappop(q) #retrieve the lowest cost next step from the queue for checking
        loc = path[-1]
                    
        if cost > max_cost:
            break #no more good paths will be found with our lowest_cost_first approaah
        
        if (tuple(loc), tuple(direction)) in visited:  # finding a path that merges with a previously approved tile?
            if cost == visited[(tuple(loc), tuple(direction))]:
                tiles[(tuple(loc), tuple(direction))].add(prev) # we still want to add a link of what tiles we visited from here
            continue

        visited[(tuple(loc), tuple(direction))] = cost #record the cost of the current tile
        tiles[(tuple(loc), tuple(direction))].add(prev)#record the connection
        prev = (tuple(loc), tuple(direction))#update the previous loc and direction
        # add possible next moves to the queue
       
        # need to explore what happens if we turn left and right, for a cost of 1000 each.
        heapq.heappush(q, (cost+1000, path, turn_left(direction), prev))
        heapq.heappush(q, (cost+1000, path, turn_right(direction), prev))
        one_ahead = [loc[0] + direction[0], loc[1]+direction[1]] #look at the space one ahead of us:
        if arr[one_ahead[0], one_ahead[1]] != "#":
            path_plus_ahead = path + [one_ahead]
            # print(path_plus_ahead)
            heapq.heappush(q, (cost+1, path_plus_ahead, direction, prev))#if the space one ahead of us is not a wall, it is an empty space or the end, and we need to explore it for a cost of 1
            #WE HAVE FOUND THE NEXT STEP AND NEED TO ADD IT TO A PATH SOMEHOW        

    return tiles

In [50]:
max_cost

82460

In [51]:
puzzle_input = aocd.get_data()
arr = parse_input(puzzle_input)

In [56]:
nodes = dijkstra_max_cost(arr, cost)

Nodes is now a list of tiles connected to the set of locs and directions we reached it through without exceeding the max cost, which means we can find the finishing tile, and walk backwards in all directions we potentially reached it through, accumulating a set of visited tiles on the way

In [93]:
tiles_visited, routes = set(), set()
def walk_backwards(position, direction):
    if type(position)!=None and tuple(position) not in routes:
        routes.add((tuple(position), tuple(direction)))
        tiles_visited.add(tuple(position))
        for new_pos in nodes[(tuple(position), tuple(direction))]:
            if new_pos:
                walk_backwards(new_pos[0], new_pos[1])


In [94]:
target = np.argwhere(arr == "E")[0]

In [95]:
target

array([  1, 139])

In [96]:
for d in dirs:
    walk_backwards(target, d)

In [97]:
len(tiles_visited)

590

TypeError: 'set' object is not subscriptable

In [ ]:
  def walk(cur):
        if cur and cur not in routes:
            routes.add(cur)
            tiles.add(cur[0])
            for npos in links[cur]: walk(npos)
    for d in range(4): walk((target, d))